In [2]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('grey_water_management.csv')

In [4]:
X = df[['TDS (mg/l)', 'Turbidity (NTU)', 'pH', 'Depth (m)', 'Flow Discharge (L/min)']].values
y = df['Filter Efficiency (%)'].values

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),  
    tf.keras.layers.Dense(1)  
])

In [7]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [8]:
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

Epoch 1/100

41/41 [==============================] - 4s 28ms/step - loss: 1651.4871 - val_loss: 1167.4336
Epoch 2/100
41/41 [==============================] - 0s 8ms/step - loss: 927.6277 - val_loss: 889.0299
Epoch 3/100
41/41 [==============================] - 0s 7ms/step - loss: 730.6947 - val_loss: 729.7864
Epoch 4/100
41/41 [==============================] - 0s 6ms/step - loss: 564.9990 - val_loss: 522.4333
Epoch 5/100
41/41 [==============================] - 0s 6ms/step - loss: 397.4063 - val_loss: 375.6396
Epoch 6/100
41/41 [==============================] - 0s 7ms/step - loss: 276.1972 - val_loss: 267.3139
Epoch 7/100
41/41 [==============================] - 0s 7ms/step - loss: 199.6159 - val_loss: 183.3362
Epoch 8/100
41/41 [==============================] - 0s 7ms/step - loss: 156.9677 - val_loss: 160.2353
Epoch 9/100
41/41 [==============================] - 0s 8ms/step - loss: 126.9011 - val_loss: 116.3512
Epoch 10/100
41/41 [==============================] - 0s 7ms/step - l

In [9]:
test_loss = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}")

13/13 [==============================] - 0s 3ms/step - loss: 27.5443
Test Loss: 27.54433822631836


In [10]:
y_pred = model.predict(X_test)

13/13 [==============================] - 0s 3ms/step


In [11]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('efficiency.tflite', 'wb') as f:
    f.write(tflite_model)

print("TensorFlow Lite model saved successfully.")

INFO:tensorflow:Assets written to: C:\Users\RYZEN-5\AppData\Local\Temp\tmp6y17lg7a\assets


INFO:tensorflow:Assets written to: C:\Users\RYZEN-5\AppData\Local\Temp\tmp6y17lg7a\assets


TensorFlow Lite model saved successfully.


In [13]:
interpreter = tf.lite.Interpreter(model_path="efficiency.tflite")
interpreter.allocate_tensors()

In [14]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [15]:
X_test = X_test.astype(np.float32)

In [16]:
y_pred_tflite = []
for i in range(len(X_test)):
    interpreter.set_tensor(input_details[0]['index'], [X_test[i]])
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    y_pred_tflite.append(output_data[0])

In [17]:
y_pred_tflite = np.array(y_pred_tflite)

In [19]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
mse = mean_squared_error(y_test, y_pred_tflite)
mae = mean_absolute_error(y_test, y_pred_tflite)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")

Mean Squared Error: 27.544335668842397
Mean Absolute Error: 4.415082051444348
